In [1]:
#CONTCAR:A completed reaction pathway structure (substrate + surface molecule).
#BASECAR:New catalyst substrate model files to be constructed.
#POSCAR: A new model for catalyst surface calculations.
import os
import shutil
import pymatgen.core as mg
import pandas as pd
from pymatgen.io.vasp import Poscar

In [2]:
Hight_max = 0.43 #Divided height of the model surface in Cartesian coordinate
site_old_Species = 'Cu'#Location of the active site of the original structure
site_old_num = 1 #Active site number of the original structure
site_new_Species = 'Cu'#Location of the active site of the new structure
site_new_num = 1 #Active site number of the new structure
Species_df = pd.DataFrame(columns=['H','C','N','O','Cu'])#Order of possible elements
Species_element = Species_df.columns.values.tolist()

base = Poscar.from_file("BASE")
base = base.structure
Species_2 = 0
base_num = 0
base_species = 'H'

for i in range(base.num_sites):
    if base_species != str(base.species[i]):
        base_species = str(base.species[i])
        Species_df.loc['Start_index',base_species] = i
    if site_new_Species == str(base.species[i]):
        Species_2 = Species_2 + 1
        if site_new_num == Species_2:
            site_B_index = i
            break
           
for i in range(Species_df.shape[1]):
    if Species_df.iloc[0][i] != Species_df.iloc[0][i]:
        Species_df.iloc[0][i] = Species_df.iloc[0][i+1]
for i in range(Species_df.shape[1]):
    if i != Species_df.shape[1]-1:
        Species_df.iloc[0][i] = Species_df.iloc[0][i+1]
    else:
        Species_df.iloc[0][i] = base.num_sites
        
Species_df_s = Species_df.copy(deep=True)        

In [3]:
for root, dirs, files in os.walk(os.getcwd()):
        dir_list = dirs
        break
dir_list.remove('.ipynb_checkpoints')

for dirs in dir_list:
    base = Poscar.from_file("BASE")
    base = base.structure
    Species_df = Species_df_s.copy(deep=True)    
    #Reading structure files
    old_str = Poscar.from_file(dirs+'/'+"CONTCAR")
    old_str = old_str.structure
    #Determine the displacement vector between the two active sites
    Species_1 = 0
    for i in range(old_str.num_sites):
        if site_old_Species == str(old_str.species[i]):
            Species_1 = Species_1 + 1
            if site_old_num == Species_1:
                site_A_index = i
                break
    Displacement_vector = base.frac_coords[site_B_index] - old_str.frac_coords[site_A_index]
    num = 0
    for i in range(old_str.num_sites):
        coord_1 = old_str.frac_coords[i-num]
        if coord_1[2] < Hight_max :
            old_str.remove_sites([i-num])
            num = num + 1
    #Removal of the base atoms from the original structure file
    num = 0
    for i in range(old_str.num_sites):
        coord_1 = old_str.frac_coords[i-num]
        if coord_1[2] < Hight_max :
            old_str.remove_sites([i-num])
            num = num + 1
    #Adding molecules to a new substrate file
    for i in range(old_str.num_sites):
        insert_index = Species_df.loc['Start_index',str(old_str.species[i])]
        old_spec = old_str.species[i]
        coord_new = old_str.frac_coords[i] + Displacement_vector
        base.insert(insert_index,old_spec,coord_new)
        for m in range(Species_element.index(str(old_str.species[i])),Species_df.shape[1]):
            Species_df.iloc[0,m] = Species_df.iloc[0,m] + 1
    #Exporting a new structure file
    base.to(dirs+'/'+"POSCAR")
    shutil.copyfile('INCAR',dirs+'/'+"INCAR")
    shutil.copyfile('KPOINTS',dirs+'/'+"KPOINTS")

/opt/anaconda3/envs/VASP/lib/python3.11/site-packages/pymatgen/core/structure.py:756: UserWarning: Not all sites have property velocities. Missing values are set to None.
  warnings.warn(f"Not all sites have property {k}. Missing values are set to None.")
/opt/anaconda3/envs/VASP/lib/python3.11/site-packages/pymatgen/io/vasp/inputs.py:507: UserWarning: Velocities are missing or corrupted.
  warnings.warn("Velocities are missing or corrupted.")
